In [ ]:
# ref : https://www.kaggle.com/datasnaek/youtube-new
# ref2 : https://www.kaggle.com/rsrishav/youtube-trending-video-dataset

# 한국어 전처리 : https://signing.tistory.com/74

### import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install googletrans==4.0.0-rc1

     |████████████████████████████████| 55 kB 2.6 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 1.3 MB 22.1 MB/s 
     |████████████████████████████████| 65 kB 3.4 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17415 sha256=026c43307f95322dffe3843b2b450a00abf21250db81dc6db115a9f8efb7a5a8
  Stored in directory: /root/.cache/pip/wheels/43/34/00/4fe71786ea6d12314b29037620c36d857e5d104ac2748bf82a
Successfully built googletrans


In [3]:
# !pip install Selenium
# !apt-get update # to update ubuntu to correctly run apt install
# !apt install chromium-chromedriver

In [3]:
# Listing the directories with datasets
import os

#Importing dependencies/packages

# Utility dependencies
import re
import pickle

# Graphic/math dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud

import time

from tqdm.auto import tqdm

In [4]:
### nltk download
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords

from googletrans import Translator

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### 재생 안되는 영상 제거

예외 처리가 제대로 되지 않아 로컬에서 수행하였습니다

In [6]:
# from selenium import webdriver
# from selenium.common.exceptions import NoSuchElementException
# from selenium.webdriver.common.by import By
# import time

# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')

# driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

# can_play = 0
# can_play_list = []
# for video in tqdm(data['video_id']):
#   driver.get(f'https://www.youtube.com/watch?v={video}')
#   time.sleep(3)
#   try:
#       tag = driver.find_element(By.CSS_SELECTOR, 'body > ytd-app > #content > #page-manager > ytd-watch-flexy > #columns > #primary > #primary-inner > #player > #error-screen')
#       print(tag)
#       print(f'재생 불가: {video}')
#   except NoSuchElementException:
#       print(f'재생 가능 : {video}')
#       can_play += 1
#       can_play_list.append(video)
# print(can_play)
# print(can_play_list)

## Korean

### data : youtube-trending-video-dataset KR

In [5]:
data_kr1 = pd.read_feather('/content/drive/MyDrive/Capstone/data/KR_youtube_trending_data_able.ftr')

In [6]:
data_kr1.head()

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,uq5LClQN3cE,안녕하세요 보겸입니다,2020-08-09T09:32:48Z,UCu9BCtGIEr73LXZsKmoujKw,보겸 BK,24,2020-08-12T00:00:00Z,보겸|bokyem,5947503,53326,105756,139946,https://i.ytimg.com/vi/uq5LClQN3cE/default.jpg,False,False,None
1,I-ZbZCHsHD0,부락토스의 계획 [총몇명 프리퀄],2020-08-12T09:00:08Z,UCRuSxVu4iqTK5kCh90ntAgA,총몇명,1,2020-08-12T00:00:00Z,총몇명|재밌는 만화|부락토스|루시퍼|총몇명 프리퀄|총몇명 스토리,963384,28244,494,3339,https://i.ytimg.com/vi/I-ZbZCHsHD0/default.jpg,False,False,"오늘도 정말 감사드립니다!!총몇명 스튜디오 - 총몇명, 십제곱, 5G민, MOVE혁..."
2,3pI_L3-sMVg,안녕하세요 꽈뚜룹입니다.,2020-08-11T15:00:58Z,UCkQCwnkQfgSuPTTnw_Y7v7w,꽈뚜룹 Quaddurup,24,2020-08-12T00:00:00Z,꽈뚜룹|한국여행기|quaddurup|뚜룹이|korea|southkorea|vlog|...,1743374,36893,1798,8751,https://i.ytimg.com/vi/3pI_L3-sMVg/default.jpg,False,False,앞으로 좀 더 깔끔한 영상제작 약속 드리겠습니다.늘 감사드립니다
3,zrsBjYukE8s,박진영 (J.Y. Park) When We Disco (Duet with 선미) M/V,2020-08-11T09:00:13Z,UCaO6TYtlC8U5ttz62hTrZgg,JYP Entertainment,10,2020-08-12T00:00:00Z,JYP Entertainment|JYP|J.Y.Park|JYPark|박진영|선미|S...,3433885,353337,9763,23405,https://i.ytimg.com/vi/zrsBjYukE8s/default.jpg,False,False,MelOn http://kko.to/TWyXd7zYjSpotify https://s...
4,jbGRowa5tIk,ITZY “Not Shy” M/V TEASER,2020-08-11T15:00:13Z,UCaO6TYtlC8U5ttz62hTrZgg,JYP Entertainment,10,2020-08-12T00:00:00Z,JYP Entertainment|JYP|ITZY|있지|ITZY Video|ITZY ...,6000070,714301,15176,31040,https://i.ytimg.com/vi/jbGRowa5tIk/default.jpg,False,False,ITZY Not Shy M/V[ITZY Official] https://www.yo...


In [7]:
data_kr1.shape

(11540, 16)

In [8]:
data_kr1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11540 entries, 0 to 11539
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   video_id           11540 non-null  object
 1   title              11540 non-null  object
 2   publishedAt        11540 non-null  object
 3   channelId          11540 non-null  object
 4   channelTitle       11540 non-null  object
 5   categoryId         11540 non-null  int64 
 6   trending_date      11540 non-null  object
 7   tags               11540 non-null  object
 8   view_count         11540 non-null  int64 
 9   likes              11540 non-null  int64 
 10  dislikes           11540 non-null  int64 
 11  comment_count      11540 non-null  int64 
 12  thumbnail_link     11540 non-null  object
 13  comments_disabled  11540 non-null  bool  
 14  ratings_disabled   11540 non-null  bool  
 15  description        11303 non-null  object
dtypes: bool(2), int64(5), object(9)
memory u

### data : Trending YouTube Video Statistics US

In [9]:
data_kr2 = pd.read_feather('/content/drive/MyDrive/Capstone/data/KRvideos_able.ftr')

In [10]:
data_kr2.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,RxGQe4EeEpA,17.14.11,좋아 by 민서_윤종신_좋니 답가,라푸마코리아,22,2017-11-13T07:07:36.000Z,"라푸마|""윤종신""|""좋니""|""좋아""|""샬레""|""민서""",156130,1422,40,272,https://i.ytimg.com/vi/RxGQe4EeEpA/default.jpg,False,False,False,윤종신 '좋니'의 답가 '좋아' 최초 공개!\n그 여자의 이야기를 지금 만나보세요....
1,9V8bnWUmE9U,17.14.11,나몰라패밀리 운동화 영상 2탄 (빼빼로데이버젼),나몰라패밀리 핫쇼,22,2017-11-11T07:16:08.000Z,"아디다스|""빼빼로""|""핫쇼""|""나몰라패밀리""|""대학로""|""공연""",421409,5112,166,459,https://i.ytimg.com/vi/9V8bnWUmE9U/default.jpg,False,False,False,퍼가실때 꼭 출처 부탁드려요
2,0_8py-t5R80,17.14.11,"이명박 출국 현장, 놓치면 안되는 장면",미디어몽구,25,2017-11-12T11:19:52.000Z,"이명박|""이명박 출국금지""|""이명박 출국현장""|""이명박 바레인""|""이명박 대통령""|...",222850,2093,173,1219,https://i.ytimg.com/vi/0_8py-t5R80/default.jpg,False,False,False,다스는 누구겁니까 듣고 난 후 표정
3,bk55RbxiQdI,17.14.11,김장겸은 물러갔다 MBC 노조 환호와 눈물,NocutV,25,2017-11-13T11:08:59.000Z,"nocutV|""노컷V""|""CBS""|""mbc""|""김장겸""|""해임""|""허일후""",84466,1094,109,450,https://i.ytimg.com/vi/bk55RbxiQdI/default.jpg,False,False,False,김장겸 MBC 사장이 결국 해임됐다.\n\nMBC 대주주인 방송문화진흥회는 13일 ...
4,AmP0ryzDmbY,17.14.11,김정숙 여사는 왜 갑자기 문재인 대통령 주머니에 손을 넣었나? 인도네시아 대통령도 깜놀,하우스,25,2017-11-12T10:17:48.000Z,"김정숙|""김정숙 여사""|""김정숙 문재인""|""문재인 대통령""|""문재인""|""문재인 인도...",188707,545,311,357,https://i.ytimg.com/vi/AmP0ryzDmbY/default.jpg,False,False,False,김정숙 여사는 왜 갑자기 문재인 대통령 주머니에 손을 넣었나? 인도네시아 대통령도 ...


In [11]:
data_kr2.shape

(8388, 16)

In [12]:
data_kr2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8388 entries, 0 to 8387
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   video_id                8388 non-null   object
 1   trending_date           8388 non-null   object
 2   title                   8388 non-null   object
 3   channel_title           8388 non-null   object
 4   category_id             8388 non-null   int64 
 5   publish_time            8388 non-null   object
 6   tags                    8388 non-null   object
 7   views                   8388 non-null   int64 
 8   likes                   8388 non-null   int64 
 9   dislikes                8388 non-null   int64 
 10  comment_count           8388 non-null   int64 
 11  thumbnail_link          8388 non-null   object
 12  comments_disabled       8388 non-null   bool  
 13  ratings_disabled        8388 non-null   bool  
 14  video_error_or_removed  8388 non-null   bool  
 15  desc

### 두 dataframe 합치기

#### kr1 dataframe 전처리

In [13]:
data_kr1.rename(columns = {"publishedAt" : "publish_time", "channelTitle" : "channel_title", "categoryId" : "category_id", "view_count" : "views"}, inplace = True)

In [14]:
data_kr1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11540 entries, 0 to 11539
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   video_id           11540 non-null  object
 1   title              11540 non-null  object
 2   publish_time       11540 non-null  object
 3   channelId          11540 non-null  object
 4   channel_title      11540 non-null  object
 5   category_id        11540 non-null  int64 
 6   trending_date      11540 non-null  object
 7   tags               11540 non-null  object
 8   views              11540 non-null  int64 
 9   likes              11540 non-null  int64 
 10  dislikes           11540 non-null  int64 
 11  comment_count      11540 non-null  int64 
 12  thumbnail_link     11540 non-null  object
 13  comments_disabled  11540 non-null  bool  
 14  ratings_disabled   11540 non-null  bool  
 15  description        11303 non-null  object
dtypes: bool(2), int64(5), object(9)
memory u

In [15]:
# column 제거
data_kr1.drop(columns = ['channelId'], inplace = True)

In [16]:
data_kr1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11540 entries, 0 to 11539
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   video_id           11540 non-null  object
 1   title              11540 non-null  object
 2   publish_time       11540 non-null  object
 3   channel_title      11540 non-null  object
 4   category_id        11540 non-null  int64 
 5   trending_date      11540 non-null  object
 6   tags               11540 non-null  object
 7   views              11540 non-null  int64 
 8   likes              11540 non-null  int64 
 9   dislikes           11540 non-null  int64 
 10  comment_count      11540 non-null  int64 
 11  thumbnail_link     11540 non-null  object
 12  comments_disabled  11540 non-null  bool  
 13  ratings_disabled   11540 non-null  bool  
 14  description        11303 non-null  object
dtypes: bool(2), int64(5), object(8)
memory usage: 1.2+ MB


#### kr2 dataframe 전처리

In [17]:
data_kr2.shape

(8388, 16)

In [18]:
# error 혹은 제거된 영상 미포함
data_kr2 = data_kr2[data_kr2['video_error_or_removed'] == False]
# column 제거
data_kr2.drop(columns = ['video_error_or_removed'], inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [19]:
data_kr2.shape

(8377, 15)

In [20]:
data_kr2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8377 entries, 0 to 8387
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   video_id           8377 non-null   object
 1   trending_date      8377 non-null   object
 2   title              8377 non-null   object
 3   channel_title      8377 non-null   object
 4   category_id        8377 non-null   int64 
 5   publish_time       8377 non-null   object
 6   tags               8377 non-null   object
 7   views              8377 non-null   int64 
 8   likes              8377 non-null   int64 
 9   dislikes           8377 non-null   int64 
 10  comment_count      8377 non-null   int64 
 11  thumbnail_link     8377 non-null   object
 12  comments_disabled  8377 non-null   bool  
 13  ratings_disabled   8377 non-null   bool  
 14  description        7907 non-null   object
dtypes: bool(2), int64(5), object(8)
memory usage: 932.6+ KB


In [21]:
# 두 dataframe 병합
df_kr = pd.concat([data_kr1, data_kr2], axis = 0)
df_kr

,video_id,title,publish_time,channel_title,category_id,trending_date,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,uq5LClQN3cE,안녕하세요 보겸입니다,2020-08-09T09:32:48Z,보겸 BK,24,2020-08-12T00:00:00Z,보겸|bokyem,5947503,53326,105756,139946,https://i.ytimg.com/vi/uq5LClQN3cE/default.jpg,False,False,None
1,I-ZbZCHsHD0,부락토스의 계획 [총몇명 프리퀄],2020-08-12T09:00:08Z,총몇명,1,2020-08-12T00:00:00Z,총몇명|재밌는 만화|부락토스|루시퍼|총몇명 프리퀄|총몇명 스토리,963384,28244,494,3339,https://i.ytimg.com/vi/I-ZbZCHsHD0/default.jpg,False,False,"오늘도 정말 감사드립니다!!총몇명 스튜디오 - 총몇명, 십제곱, 5G민, MOVE혁..."
2,3pI_L3-sMVg,안녕하세요 꽈뚜룹입니다.,2020-08-11T15:00:58Z,꽈뚜룹 Quaddurup,24,2020-08-12T00:00:00Z,꽈뚜룹|한국여행기|quaddurup|뚜룹이|korea|southkorea|vlog|...,1743374,36893,1798,8751,https://i.ytimg.com/vi/3pI_L3-sMVg/default.jpg,False,False,앞으로 좀 더 깔끔한 영상제작 약속 드리겠습니다.늘 감사드립니다
3,zrsBjYukE8s,박진영 (J.Y. Park) When We Disco (Duet with 선미) M/V,2020-08-11T09:00:13Z,JYP Entertainment,10,2020-08-12T00:00:00Z,JYP Entertainment|JYP|J.Y.Park|JYPark|박진영|선미|S...,3433885,353337,9763,23405,https://i.ytimg.com/vi/zrsBjYukE8s/default.jpg,False,False,MelOn http://kko.to/TWyXd7zYjSpotify https://s...
4,jbGRowa5tIk,ITZY “Not Shy” M/V TEASER,2020-08-11T15:00:13Z,JYP Entertainment,10,2020-08-12T00:00:00Z,JYP Entertainment|JYP|ITZY|있지|ITZY Video|ITZY ...,6000070,714301,15176,31040,https://i.ytimg.com/vi/jbGRowa5tIk/default.jpg,False,False,ITZY Not Shy M/V[ITZY Official] https://www.yo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8383,RG2aK0EEaEs,초밥 몇 접시 먹었을까? 먹방 Mukbang eating show,2018-06-12T09:05:17.000Z,프란,24,18.14.06,[none],133621,1821,147,308,https://i.ytimg.com/vi/RG2aK0EEaEs/default.jpg,False,False,소음 너무나 죄송합니다. 거의 2~3초 간격으로 소음이 나서 편집으로도 감당이 안되...
8384,dWK5g_FNZ9I,[벼락맞은 문방구2] 3-1화 정체불명 괴한의 습격!,2018-06-13T06:00:01.000Z,Tooniverse-투니버스,24,18.14.06,"벼락맞은 문방구|""문방구""|""벼락""|""벼락 문방구""|""투니버스""|""toonivers...",13175,205,10,59,https://i.ytimg.com/vi/dWK5g_FNZ9I/default.jpg,False,False,벼락맞은 문방구2 정체불명 괴한의 습격! 치킨을 기다리고 있던 담임선생님과 한열이...
8385,D5slHtZSvjY,귀염뽀짝 복숭아 🍑 투명액괴만들기 | 곤약젤리 액괴 | 말랑쫀득질긴 | 슬라임 | 뿌직,2018-06-13T03:47:20.000Z,PPUJIG뿌직,22,18.14.06,[none],224994,9898,330,8620,https://i.ytimg.com/vi/D5slHtZSvjY/default.jpg,False,False,영상 찍고나서 액괴가 더 좋아졌어요 😂항상 영상 찍고 편집하고나면 더 좋아지는 액괴...
8386,C2Aw5HoDJX0,무서운이야기 24화 '미술실 자화상 귀신' 오싹튜브 학교괴담 뽀로로 장난감 애니메이...,2018-06-13T00:00:01.000Z,오싹튜브,1,18.14.06,"뽀로로|""무서운이이야기""|""장난감""|""뽀로로장난감애니메이션""|""애니메이션""|""만화""...",23404,190,29,111,https://i.ytimg.com/vi/C2Aw5HoDJX0/default.jpg,False,False,안녕하세요 ~ 오늘은 무서운 학교괴담입니다! 여러분들 학교에 어떤 괴담이 있는지 댓...


In [22]:
df_kr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19917 entries, 0 to 8387
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   video_id           19917 non-null  object
 1   title              19917 non-null  object
 2   publish_time       19917 non-null  object
 3   channel_title      19917 non-null  object
 4   category_id        19917 non-null  int64 
 5   trending_date      19917 non-null  object
 6   tags               19917 non-null  object
 7   views              19917 non-null  int64 
 8   likes              19917 non-null  int64 
 9   dislikes           19917 non-null  int64 
 10  comment_count      19917 non-null  int64 
 11  thumbnail_link     19917 non-null  object
 12  comments_disabled  19917 non-null  bool  
 13  ratings_disabled   19917 non-null  bool  
 14  description        19210 non-null  object
dtypes: bool(2), int64(5), object(8)
memory usage: 2.2+ MB


In [23]:
# 중복 영상 제거(중복 영상 없음)
df_kr.drop_duplicates(subset=['video_id'], inplace = True, ignore_index = True)
df_kr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19917 entries, 0 to 19916
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   video_id           19917 non-null  object
 1   title              19917 non-null  object
 2   publish_time       19917 non-null  object
 3   channel_title      19917 non-null  object
 4   category_id        19917 non-null  int64 
 5   trending_date      19917 non-null  object
 6   tags               19917 non-null  object
 7   views              19917 non-null  int64 
 8   likes              19917 non-null  int64 
 9   dislikes           19917 non-null  int64 
 10  comment_count      19917 non-null  int64 
 11  thumbnail_link     19917 non-null  object
 12  comments_disabled  19917 non-null  bool  
 13  ratings_disabled   19917 non-null  bool  
 14  description        19210 non-null  object
dtypes: bool(2), int64(5), object(8)
memory usage: 2.0+ MB


In [24]:
df_kr.reset_index(drop = True, inplace = True)

### 전처리 및 학습데이터 생성

#### 영상 제목

In [25]:
df_kr.title.value_counts()

브베 실시간 스트리밍                                                                                12
Go Back (고백)                                                                                4
안녕하세요 보겸입니다                                                                                 3
가족                                                                                          2
BTS (방탄소년단) Happy Chuseok Greeting                                                          2
                                                                                           ..
[ENG] Show Me The Money 9 [SMTM9] 본격 프로듀서 뒷담화?! 대기실에서 과연 무슨 일이.. I 10월 첫 방송 200101 EP.0     1
XIA (준수) - 사랑하고 싶지 않아 (바른연애 길잡이 X XIA (준수)) 가로라이브 Full ver.                                 1
식탁 위에 / 절대 두면 안될 / 물건 5가지                                                                   1
(ENG) 여사친이 내 무릎에 누워버렸다 [잘 하고 싶어] - EP.08                                                    1
계란말이 이렇게 해보세요, 새로운 맛에 잊을 수 없는 비주얼입니다                        

#### 영상 키워드

In [26]:
df_kr.tags.value_counts()

[None]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     1419
[none]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [27]:
# tags 결측치 처리
df_kr.loc[df_kr['tags'].isin(['[None]', '[none]']), 'tags'] = 'None'
df_kr.tags.value_counts()

None                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       2730
짤툰|만화|애니메이션|웹툰|병맛|더빙|병맛더빙|yt:cc=on                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [28]:
# 키워드를 list로 변형
df_kr['tags_list'] = df_kr['tags'].str.split('|')
df_kr['tags_list']

0                                             [보겸, bokyem]
1               [총몇명, 재밌는 만화, 부락토스, 루시퍼, 총몇명 프리퀄, 총몇명 스토리]
2        [꽈뚜룹, 한국여행기, quaddurup, 뚜룹이, korea, southkorea...
3        [JYP Entertainment, JYP, J.Y.Park, JYPark, 박진영...
4        [JYP Entertainment, JYP, ITZY, 있지, ITZY Video,...
                               ...                        
19912                                               [None]
19913    [벼락맞은 문방구, "문방구", "벼락", "벼락 문방구", "투니버스", "too...
19914                                               [None]
19915    [뽀로로, "무서운이이야기", "장난감", "뽀로로장난감애니메이션", "애니메이션"...
19916    [YTN, "YTN 뉴스", "지방선거", "613지방선거", "선거", "개표",...
Name: tags_list, Length: 19917, dtype: object

In [29]:
# 키워드 list를 문장으로 concat
df_kr['tagstring'] = df_kr['tags_list'].apply(lambda x: ' '.join(map(str, x)))
df_kr['tagstring']

0                                                보겸 bokyem
1                      총몇명 재밌는 만화 부락토스 루시퍼 총몇명 프리퀄 총몇명 스토리
2        꽈뚜룹 한국여행기 quaddurup 뚜룹이 korea southkorea vlog ...
3        JYP Entertainment JYP J.Y.Park JYPark 박진영 선미 S...
4        JYP Entertainment JYP ITZY 있지 ITZY Video ITZY ...
                               ...                        
19912                                                 None
19913    벼락맞은 문방구 "문방구" "벼락" "벼락 문방구" "투니버스" "toonivers...
19914                                                 None
19915    뽀로로 "무서운이이야기" "장난감" "뽀로로장난감애니메이션" "애니메이션" "만화"...
19916    YTN "YTN 뉴스" "지방선거" "613지방선거" "선거" "개표" "개표방송"...
Name: tagstring, Length: 19917, dtype: object

In [30]:
# ("는 추후 전처리)
df_kr['tagstring'].value_counts()

None                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       2730
짤툰 만화 애니메이션 웹툰 병맛 더빙 병맛더빙 yt:cc=on                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [31]:
# 키워드 문장 이외 column 제거
df_kr.drop(columns = ['tags', 'tags_list'], inplace = True)
df_kr.head()

,video_id,title,publish_time,channel_title,category_id,trending_date,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,tagstring
0,uq5LClQN3cE,안녕하세요 보겸입니다,2020-08-09T09:32:48Z,보겸 BK,24,2020-08-12T00:00:00Z,5947503,53326,105756,139946,https://i.ytimg.com/vi/uq5LClQN3cE/default.jpg,False,False,None,보겸 bokyem
1,I-ZbZCHsHD0,부락토스의 계획 [총몇명 프리퀄],2020-08-12T09:00:08Z,총몇명,1,2020-08-12T00:00:00Z,963384,28244,494,3339,https://i.ytimg.com/vi/I-ZbZCHsHD0/default.jpg,False,False,"오늘도 정말 감사드립니다!!총몇명 스튜디오 - 총몇명, 십제곱, 5G민, MOVE혁...",총몇명 재밌는 만화 부락토스 루시퍼 총몇명 프리퀄 총몇명 스토리
2,3pI_L3-sMVg,안녕하세요 꽈뚜룹입니다.,2020-08-11T15:00:58Z,꽈뚜룹 Quaddurup,24,2020-08-12T00:00:00Z,1743374,36893,1798,8751,https://i.ytimg.com/vi/3pI_L3-sMVg/default.jpg,False,False,앞으로 좀 더 깔끔한 영상제작 약속 드리겠습니다.늘 감사드립니다,꽈뚜룹 한국여행기 quaddurup 뚜룹이 korea southkorea vlog ...
3,zrsBjYukE8s,박진영 (J.Y. Park) When We Disco (Duet with 선미) M/V,2020-08-11T09:00:13Z,JYP Entertainment,10,2020-08-12T00:00:00Z,3433885,353337,9763,23405,https://i.ytimg.com/vi/zrsBjYukE8s/default.jpg,False,False,MelOn http://kko.to/TWyXd7zYjSpotify https://s...,JYP Entertainment JYP J.Y.Park JYPark 박진영 선미 S...
4,jbGRowa5tIk,ITZY “Not Shy” M/V TEASER,2020-08-11T15:00:13Z,JYP Entertainment,10,2020-08-12T00:00:00Z,6000070,714301,15176,31040,https://i.ytimg.com/vi/jbGRowa5tIk/default.jpg,False,False,ITZY Not Shy M/V[ITZY Official] https://www.yo...,JYP Entertainment JYP ITZY 있지 ITZY Video ITZY ...


#### 영상 내용

In [32]:
df_kr.description.value_counts()

Deweni Inima\nwatch all Episodes :- https://goo.gl/svIGex\n------------------------------------------------------------------------------------\nWatch Sinhala Teledramas, gossips, pictures & get more updates...\nhttp://www.derana.lk\nhttp://derana.lk/dreamstar\nhttp://derana.lk/misssrilanka\nJoin official fan page :- http://www.facebook.com/tvderanaofficial\nAndroid App for Mobile :- http://goo.gl/KMMu1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

#### 감정 예측 데이터 생성

##### description 추가

In [33]:
# 영상 내용 추가
desc_null, desc = df_kr[df_kr['description'].isnull()].index, df_kr[df_kr['description'].notnull()].index
df_kr.loc[desc_null, 'pred'] = 'None'
df_kr.loc[desc, 'pred'] = df_kr['description']

In [34]:
df_kr.loc[desc_null, :].head()

,video_id,title,publish_time,channel_title,category_id,trending_date,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,tagstring,pred
0,uq5LClQN3cE,안녕하세요 보겸입니다,2020-08-09T09:32:48Z,보겸 BK,24,2020-08-12T00:00:00Z,5947503,53326,105756,139946,https://i.ytimg.com/vi/uq5LClQN3cE/default.jpg,False,False,None,보겸 bokyem,None
5,REUA4roJndU,진심으로 사과드립니다. 죄송합니다.,2020-08-11T14:00:01Z,임다TV,24,2020-08-12T00:00:00Z,659743,13040,13337,18286,https://i.ytimg.com/vi/REUA4roJndU/default.jpg,False,False,None,None,None
66,mEAIdRGCIY0,한말씀 더 드리겠습니다,2020-08-08T11:54:57Z,보겸 BK,24,2020-08-12T00:00:00Z,2836691,54459,21528,28926,https://i.ytimg.com/vi/mEAIdRGCIY0/default.jpg,False,False,None,보겸 bokyem,None
75,nlx-H5kxxkw,정동원 남승민&임도형 웰컴 투 하동 집들이 [아내의 맛],2020-08-05T02:43:53Z,정동원TV,10,2020-08-12T00:00:00Z,1815842,31481,1798,2186,https://i.ytimg.com/vi/nlx-H5kxxkw/default.jpg,False,False,None,None,None
159,VlJmXPTimNU,08/13 존리대표 Q&A 한국 주식 지금 사도 좋을까?,2020-08-13T09:20:47Z,존리라이프스타일 주식,27,2020-08-14T00:00:00Z,56262,1691,55,123,https://i.ytimg.com/vi/VlJmXPTimNU/default.jpg,False,False,None,금융 금융교육 금융문맹 금융투자 주식 주식투자 주식교육 펀드 펀드투자 투자 투자교육...,None


In [35]:
df_kr.loc[desc, :].head()

,video_id,title,publish_time,channel_title,category_id,trending_date,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,tagstring,pred
1,I-ZbZCHsHD0,부락토스의 계획 [총몇명 프리퀄],2020-08-12T09:00:08Z,총몇명,1,2020-08-12T00:00:00Z,963384,28244,494,3339,https://i.ytimg.com/vi/I-ZbZCHsHD0/default.jpg,False,False,"오늘도 정말 감사드립니다!!총몇명 스튜디오 - 총몇명, 십제곱, 5G민, MOVE혁...",총몇명 재밌는 만화 부락토스 루시퍼 총몇명 프리퀄 총몇명 스토리,"오늘도 정말 감사드립니다!!총몇명 스튜디오 - 총몇명, 십제곱, 5G민, MOVE혁..."
2,3pI_L3-sMVg,안녕하세요 꽈뚜룹입니다.,2020-08-11T15:00:58Z,꽈뚜룹 Quaddurup,24,2020-08-12T00:00:00Z,1743374,36893,1798,8751,https://i.ytimg.com/vi/3pI_L3-sMVg/default.jpg,False,False,앞으로 좀 더 깔끔한 영상제작 약속 드리겠습니다.늘 감사드립니다,꽈뚜룹 한국여행기 quaddurup 뚜룹이 korea southkorea vlog ...,앞으로 좀 더 깔끔한 영상제작 약속 드리겠습니다.늘 감사드립니다
3,zrsBjYukE8s,박진영 (J.Y. Park) When We Disco (Duet with 선미) M/V,2020-08-11T09:00:13Z,JYP Entertainment,10,2020-08-12T00:00:00Z,3433885,353337,9763,23405,https://i.ytimg.com/vi/zrsBjYukE8s/default.jpg,False,False,MelOn http://kko.to/TWyXd7zYjSpotify https://s...,JYP Entertainment JYP J.Y.Park JYPark 박진영 선미 S...,MelOn http://kko.to/TWyXd7zYjSpotify https://s...
4,jbGRowa5tIk,ITZY “Not Shy” M/V TEASER,2020-08-11T15:00:13Z,JYP Entertainment,10,2020-08-12T00:00:00Z,6000070,714301,15176,31040,https://i.ytimg.com/vi/jbGRowa5tIk/default.jpg,False,False,ITZY Not Shy M/V[ITZY Official] https://www.yo...,JYP Entertainment JYP ITZY 있지 ITZY Video ITZY ...,ITZY Not Shy M/V[ITZY Official] https://www.yo...
6,7FOi40uehC4,집에 혼자 있을 때 하는 짓ㅋㅋㅋㅋㅋㅋㅋ [밍꼬발랄],2020-08-12T09:00:02Z,밍꼬발랄Mingggo,23,2020-08-12T00:00:00Z,340149,13166,338,5697,https://i.ytimg.com/vi/7FOi40uehC4/default.jpg,False,False,다~해 본 사람 손!![밍꼬 배경화면]_https://grafolio.naver.c...,밍꼬발랄 밍꼬 꼬밍 발랄 모아보기 이어보기 정주행 꿀잼 꿀팁 유형 유형별 꼭있는유형...,다~해 본 사람 손!![밍꼬 배경화면]_https://grafolio.naver.c...


In [36]:
df_kr['pred'].value_counts()

None                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

##### 키워드 추가

In [37]:
# 영상 키워드 추가
kw_null, kw = df_kr[df_kr['tagstring'] == 'None'].index, df_kr[df_kr['tagstring'] != 'None'].index
df_kr.loc[kw_null, 'pred'] = df_kr['pred']
df_kr.loc[kw, 'pred'] = df_kr['tagstring'].str.cat(df_kr['pred'], sep=" ")

In [38]:
df_kr.loc[kw_null, :].head()

,video_id,title,publish_time,channel_title,category_id,trending_date,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,tagstring,pred
5,REUA4roJndU,진심으로 사과드립니다. 죄송합니다.,2020-08-11T14:00:01Z,임다TV,24,2020-08-12T00:00:00Z,659743,13040,13337,18286,https://i.ytimg.com/vi/REUA4roJndU/default.jpg,False,False,None,None,None
25,Wl4_eeRw0JQ,안녕하세요. 공대생 입니다,2020-08-10T16:59:15Z,공대생 변승주 DS,23,2020-08-12T00:00:00Z,968225,22899,1892,4896,https://i.ytimg.com/vi/Wl4_eeRw0JQ/default.jpg,False,False,2018년 9월 20일에 올렸던 지방흡입 후기 영상은 의료 뒷광고가 아니었고 협찬사...,None,2018년 9월 20일에 올렸던 지방흡입 후기 영상은 의료 뒷광고가 아니었고 협찬사...
45,mCIEbuv2Rg4,우원재 (Woo) - 'USED TO' Official Music Video [EN...,2020-08-11T09:22:19Z,AOMGOFFICIAL,10,2020-08-12T00:00:00Z,340815,26482,114,1713,https://i.ytimg.com/vi/mCIEbuv2Rg4/default.jpg,False,False,우원재 (Woo) - 'USED TO' Official Music Video*Eng...,None,우원재 (Woo) - 'USED TO' Official Music Video*Eng...
49,b9r4NQUfgNA,김어준의 다스뵈이다 125회 이낙연 박주민 그리고 미.친.김감독2,2020-08-07T13:00:13Z,딴지방송국,25,2020-08-12T00:00:00Z,1077579,64976,1923,3929,https://i.ytimg.com/vi/b9r4NQUfgNA/default.jpg,False,False,멤버십 가입하기 : https://www.youtube.com/channel/UCx...,None,멤버십 가입하기 : https://www.youtube.com/channel/UCx...
58,-5NwkXsx4wY,살인 유전자를 물려받은 남매!,2020-08-08T04:00:30Z,달빛뮤즈,1,2020-08-12T00:00:00Z,629740,4979,270,308,https://i.ytimg.com/vi/-5NwkXsx4wY/default.jpg,False,False,이번 소개 해 드릴 드라마는#악의꽃 #달빛뮤즈 #tvN드라마▶ 달빛뮤즈 채널 구독 ...,None,이번 소개 해 드릴 드라마는#악의꽃 #달빛뮤즈 #tvN드라마▶ 달빛뮤즈 채널 구독 ...


In [39]:
df_kr.loc[kw, :].head()

,video_id,title,publish_time,channel_title,category_id,trending_date,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,tagstring,pred
0,uq5LClQN3cE,안녕하세요 보겸입니다,2020-08-09T09:32:48Z,보겸 BK,24,2020-08-12T00:00:00Z,5947503,53326,105756,139946,https://i.ytimg.com/vi/uq5LClQN3cE/default.jpg,False,False,None,보겸 bokyem,보겸 bokyem None
1,I-ZbZCHsHD0,부락토스의 계획 [총몇명 프리퀄],2020-08-12T09:00:08Z,총몇명,1,2020-08-12T00:00:00Z,963384,28244,494,3339,https://i.ytimg.com/vi/I-ZbZCHsHD0/default.jpg,False,False,"오늘도 정말 감사드립니다!!총몇명 스튜디오 - 총몇명, 십제곱, 5G민, MOVE혁...",총몇명 재밌는 만화 부락토스 루시퍼 총몇명 프리퀄 총몇명 스토리,총몇명 재밌는 만화 부락토스 루시퍼 총몇명 프리퀄 총몇명 스토리 오늘도 정말 감사드...
2,3pI_L3-sMVg,안녕하세요 꽈뚜룹입니다.,2020-08-11T15:00:58Z,꽈뚜룹 Quaddurup,24,2020-08-12T00:00:00Z,1743374,36893,1798,8751,https://i.ytimg.com/vi/3pI_L3-sMVg/default.jpg,False,False,앞으로 좀 더 깔끔한 영상제작 약속 드리겠습니다.늘 감사드립니다,꽈뚜룹 한국여행기 quaddurup 뚜룹이 korea southkorea vlog ...,꽈뚜룹 한국여행기 quaddurup 뚜룹이 korea southkorea vlog ...
3,zrsBjYukE8s,박진영 (J.Y. Park) When We Disco (Duet with 선미) M/V,2020-08-11T09:00:13Z,JYP Entertainment,10,2020-08-12T00:00:00Z,3433885,353337,9763,23405,https://i.ytimg.com/vi/zrsBjYukE8s/default.jpg,False,False,MelOn http://kko.to/TWyXd7zYjSpotify https://s...,JYP Entertainment JYP J.Y.Park JYPark 박진영 선미 S...,JYP Entertainment JYP J.Y.Park JYPark 박진영 선미 S...
4,jbGRowa5tIk,ITZY “Not Shy” M/V TEASER,2020-08-11T15:00:13Z,JYP Entertainment,10,2020-08-12T00:00:00Z,6000070,714301,15176,31040,https://i.ytimg.com/vi/jbGRowa5tIk/default.jpg,False,False,ITZY Not Shy M/V[ITZY Official] https://www.yo...,JYP Entertainment JYP ITZY 있지 ITZY Video ITZY ...,JYP Entertainment JYP ITZY 있지 ITZY Video ITZY ...


In [40]:
df_kr['pred'].value_counts()

None                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

##### 제목 추가

In [41]:
# 영상 제목 추가
pr_null, pr = df_kr[df_kr['pred'].isin(['None'])].index, df_kr[~df_kr['pred'].isin(['None'])].index,
df_kr.loc[pr_null, 'pred'] = df_kr['title']
df_kr.loc[pr, 'pred'] = df_kr['title'].str.cat(df_kr['pred'], sep=" ")

In [42]:
df_kr.loc[pr_null, :].head()

,video_id,title,publish_time,channel_title,category_id,trending_date,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,tagstring,pred
5,REUA4roJndU,진심으로 사과드립니다. 죄송합니다.,2020-08-11T14:00:01Z,임다TV,24,2020-08-12T00:00:00Z,659743,13040,13337,18286,https://i.ytimg.com/vi/REUA4roJndU/default.jpg,False,False,None,None,진심으로 사과드립니다. 죄송합니다.
75,nlx-H5kxxkw,정동원 남승민&임도형 웰컴 투 하동 집들이 [아내의 맛],2020-08-05T02:43:53Z,정동원TV,10,2020-08-12T00:00:00Z,1815842,31481,1798,2186,https://i.ytimg.com/vi/nlx-H5kxxkw/default.jpg,False,False,None,None,정동원 남승민&임도형 웰컴 투 하동 집들이 [아내의 맛]
182,RD0Ppk5UKDo,"광화문에 지금 대군중 집결, “독재타도”",2020-08-15T05:37:15Z,조갑제TV,25,2020-08-15T00:00:00Z,119417,10549,775,481,https://i.ytimg.com/vi/RD0Ppk5UKDo/default.jpg,False,False,None,None,"광화문에 지금 대군중 집결, “독재타도”"
194,f_pArUKQJUc,"광복절, 한국은행 앞은 초비상 !! 우리공화당 태극기집회 원천봉쇄.",2020-08-15T05:08:44Z,파파존 TV,24,2020-08-15T00:00:00Z,46732,4592,180,96,https://i.ytimg.com/vi/f_pArUKQJUc/default.jpg,False,False,None,None,"광복절, 한국은행 앞은 초비상 !! 우리공화당 태극기집회 원천봉쇄."
220,zovQT7Puy7I,한밤에 청와대행,2020-08-15T14:33:09Z,이봉규TV,24,2020-08-16T00:00:00Z,176063,22469,455,974,https://i.ytimg.com/vi/zovQT7Puy7I/default.jpg,False,False,None,None,한밤에 청와대행


In [43]:
df_kr.loc[pr, :].head()

,video_id,title,publish_time,channel_title,category_id,trending_date,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,tagstring,pred
0,uq5LClQN3cE,안녕하세요 보겸입니다,2020-08-09T09:32:48Z,보겸 BK,24,2020-08-12T00:00:00Z,5947503,53326,105756,139946,https://i.ytimg.com/vi/uq5LClQN3cE/default.jpg,False,False,None,보겸 bokyem,안녕하세요 보겸입니다 보겸 bokyem None
1,I-ZbZCHsHD0,부락토스의 계획 [총몇명 프리퀄],2020-08-12T09:00:08Z,총몇명,1,2020-08-12T00:00:00Z,963384,28244,494,3339,https://i.ytimg.com/vi/I-ZbZCHsHD0/default.jpg,False,False,"오늘도 정말 감사드립니다!!총몇명 스튜디오 - 총몇명, 십제곱, 5G민, MOVE혁...",총몇명 재밌는 만화 부락토스 루시퍼 총몇명 프리퀄 총몇명 스토리,부락토스의 계획 [총몇명 프리퀄] 총몇명 재밌는 만화 부락토스 루시퍼 총몇명 프리퀄...
2,3pI_L3-sMVg,안녕하세요 꽈뚜룹입니다.,2020-08-11T15:00:58Z,꽈뚜룹 Quaddurup,24,2020-08-12T00:00:00Z,1743374,36893,1798,8751,https://i.ytimg.com/vi/3pI_L3-sMVg/default.jpg,False,False,앞으로 좀 더 깔끔한 영상제작 약속 드리겠습니다.늘 감사드립니다,꽈뚜룹 한국여행기 quaddurup 뚜룹이 korea southkorea vlog ...,안녕하세요 꽈뚜룹입니다. 꽈뚜룹 한국여행기 quaddurup 뚜룹이 k...
3,zrsBjYukE8s,박진영 (J.Y. Park) When We Disco (Duet with 선미) M/V,2020-08-11T09:00:13Z,JYP Entertainment,10,2020-08-12T00:00:00Z,3433885,353337,9763,23405,https://i.ytimg.com/vi/zrsBjYukE8s/default.jpg,False,False,MelOn http://kko.to/TWyXd7zYjSpotify https://s...,JYP Entertainment JYP J.Y.Park JYPark 박진영 선미 S...,박진영 (J.Y. Park) When We Disco (Duet with 선미) M...
4,jbGRowa5tIk,ITZY “Not Shy” M/V TEASER,2020-08-11T15:00:13Z,JYP Entertainment,10,2020-08-12T00:00:00Z,6000070,714301,15176,31040,https://i.ytimg.com/vi/jbGRowa5tIk/default.jpg,False,False,ITZY Not Shy M/V[ITZY Official] https://www.yo...,JYP Entertainment JYP ITZY 있지 ITZY Video ITZY ...,ITZY “Not Shy” M/V TEASER JYP Entertainment JY...


In [44]:
df_kr['pred'].value_counts()

브베 실시간 스트리밍 도네이션 후원 링크 입니당.https://toon.at/donate/ttuet(음성 도네이션만 받고 있어요)브베 페이스북 페이지 친구 해주세용~https://www.facebook.com/브베페이지-2074999436116625                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          9
브베 실시간 스트리밍 투네이션 후원 링크 입니당.https://toon.at/donate/ttuet(음성 도네이션만 

### 예측 데이터 전처리

#### 한국어 전처리

In [45]:
# 문자(한글, 영문)와 숫자만 남기고 특수문자 제거하기 
df_kr['preproc_pred'] = [re.sub('[^A-Za-z0-9가-힣]', ' ', s) for s in df_kr['pred']]

In [46]:
df_kr.head()

,video_id,title,publish_time,channel_title,category_id,trending_date,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,tagstring,pred,preproc_pred
0,uq5LClQN3cE,안녕하세요 보겸입니다,2020-08-09T09:32:48Z,보겸 BK,24,2020-08-12T00:00:00Z,5947503,53326,105756,139946,https://i.ytimg.com/vi/uq5LClQN3cE/default.jpg,False,False,None,보겸 bokyem,안녕하세요 보겸입니다 보겸 bokyem None,안녕하세요 보겸입니다 보겸 bokyem None
1,I-ZbZCHsHD0,부락토스의 계획 [총몇명 프리퀄],2020-08-12T09:00:08Z,총몇명,1,2020-08-12T00:00:00Z,963384,28244,494,3339,https://i.ytimg.com/vi/I-ZbZCHsHD0/default.jpg,False,False,"오늘도 정말 감사드립니다!!총몇명 스튜디오 - 총몇명, 십제곱, 5G민, MOVE혁...",총몇명 재밌는 만화 부락토스 루시퍼 총몇명 프리퀄 총몇명 스토리,부락토스의 계획 [총몇명 프리퀄] 총몇명 재밌는 만화 부락토스 루시퍼 총몇명 프리퀄...,부락토스의 계획 총몇명 프리퀄 총몇명 재밌는 만화 부락토스 루시퍼 총몇명 프리퀄...
2,3pI_L3-sMVg,안녕하세요 꽈뚜룹입니다.,2020-08-11T15:00:58Z,꽈뚜룹 Quaddurup,24,2020-08-12T00:00:00Z,1743374,36893,1798,8751,https://i.ytimg.com/vi/3pI_L3-sMVg/default.jpg,False,False,앞으로 좀 더 깔끔한 영상제작 약속 드리겠습니다.늘 감사드립니다,꽈뚜룹 한국여행기 quaddurup 뚜룹이 korea southkorea vlog ...,안녕하세요 꽈뚜룹입니다. 꽈뚜룹 한국여행기 quaddurup 뚜룹이 k...,꽈뚜룹입니다 꽈뚜룹 한국여행기 quaddurup 뚜룹이 k...
3,zrsBjYukE8s,박진영 (J.Y. Park) When We Disco (Duet with 선미) M/V,2020-08-11T09:00:13Z,JYP Entertainment,10,2020-08-12T00:00:00Z,3433885,353337,9763,23405,https://i.ytimg.com/vi/zrsBjYukE8s/default.jpg,False,False,MelOn http://kko.to/TWyXd7zYjSpotify https://s...,JYP Entertainment JYP J.Y.Park JYPark 박진영 선미 S...,박진영 (J.Y. Park) When We Disco (Duet with 선미) M...,박진영 J Y Park When We Disco Duet with 선미 M...
4,jbGRowa5tIk,ITZY “Not Shy” M/V TEASER,2020-08-11T15:00:13Z,JYP Entertainment,10,2020-08-12T00:00:00Z,6000070,714301,15176,31040,https://i.ytimg.com/vi/jbGRowa5tIk/default.jpg,False,False,ITZY Not Shy M/V[ITZY Official] https://www.yo...,JYP Entertainment JYP ITZY 있지 ITZY Video ITZY ...,ITZY “Not Shy” M/V TEASER JYP Entertainment JY...,ITZY Not Shy M V TEASER JYP Entertainment JY...


In [47]:
df_kr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19917 entries, 0 to 19916
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   video_id           19917 non-null  object
 1   title              19917 non-null  object
 2   publish_time       19917 non-null  object
 3   channel_title      19917 non-null  object
 4   category_id        19917 non-null  int64 
 5   trending_date      19917 non-null  object
 6   views              19917 non-null  int64 
 7   likes              19917 non-null  int64 
 8   dislikes           19917 non-null  int64 
 9   comment_count      19917 non-null  int64 
 10  thumbnail_link     19917 non-null  object
 11  comments_disabled  19917 non-null  bool  
 12  ratings_disabled   19917 non-null  bool  
 13  description        19210 non-null  object
 14  tagstring          19917 non-null  object
 15  pred               19917 non-null  object
 16  preproc_pred       19917 non-null  objec

#### 한국어 -> 영어

In [48]:
# 한국어 영어 번역

preproc_preds = df_kr['preproc_pred']

translator = Translator()

for i, kr in enumerate(tqdm(preproc_preds)):
    time.sleep(0.5)
    try:
        en = translator.translate(kr, src='ko', dest='en').text
        df_kr.loc[i, 'pred_en'] = en
    except:
        df_kr.loc[i, 'pred_en'] = kr

  0%|          | 0/19917 [00:00<?, ?it/s]

In [ ]:
# 한국어 영어 번역

# start = time.time()
# preds_en = kr_to_en(preproc_preds)
# print('Text processing complete.')
# print(f'It took {round(time.time() - start)} seconds to complete this task.')

In [49]:
df_kr.to_feather('/content/drive/MyDrive/Capstone/data/youtube_kr_preprocessed.ftr')

#### 학습데이터와 동일하게 전처리 진행

1. None 제거
2. 소문자 변환
3. URLs, Emojis, Usernames 전처리
4. 알파벳 아닌 문자 제거
5. 3번 이상 반복되는 문자 2개짜리 문자로 변환
6. 길이 1인 단어 제거
7. stop word 제거
8. 표제어 추출

In [50]:
# Extract the 'pred' column from the dataset
preds = df_kr['pred_en']

# Defining dictionary containing all emojis with their meanings.
emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}

## Defining set containing all stopwords in english.
stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from', 
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're',
             's', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']

stopword_nltk = stopwords.words('english')

def preprocess(textdata):
    processedText = []
    # wordcount={}
    
    # Create Lemmatizer and Stemmer.
    wordLemm = WordNetLemmatizer()
    
    # Defining regex patterns.
    urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
    userPattern       = '@[^\s]+'
    alphaPattern      = "[^a-zA-Z0-9]"
    sequencePattern   = r"(.)\1\1+"
    seqReplacePattern = r"\1\1"
    
    for tweet in tqdm(textdata):
        # None 제거
        tweet = tweet.replace('None', '')

        tweet = tweet.lower()

        # Replace all URls with 'URL'
        # tweet = re.sub(urlPattern,' URL',tweet)
        tweet = re.sub(urlPattern,' URL', tweet)
        # Replace all emojis.
        for emoji in emojis.keys():
            tweet = tweet.replace(emoji, "EMOJI" + emojis[emoji])        
        # Replace @USERNAME to 'USER'.
        tweet = re.sub(userPattern,' USER', tweet)        
        # Replace all non alphabets.
        tweet = re.sub(alphaPattern, " ", tweet)
        # Replace 3 or more consecutive letters by 2 letter.
        tweet = re.sub(sequencePattern, seqReplacePattern, tweet)

        tweetwords = ''
        for word in tweet.split():
            # Checking if the word is a stopword.
            if word not in stopword_nltk:
                if len(word)>1:
                    # Lemmatizing the word.
                    word = wordLemm.lemmatize(word)

                    # # 단어 나온 횟수 체크
                    # if word not in wordcount.keys():
                    #     wordcount[word] = 1
                    # else:
                    #     wordcount[word] += 1

                    tweetwords += (word+' ')
                    
        processedText.append(tweetwords)

    return processedText

In [51]:
# Preprocessing the texts

start = time.time()
processed_preds = preprocess(preds)
print('Text processing complete.')
print(f'It took {round(time.time() - start)} seconds to complete this task.')

  0%|          | 0/19917 [00:00<?, ?it/s]

Text processing complete.
It took 16 seconds to complete this task.


In [52]:
df_kr['preprocessed_pred'] = processed_preds

In [53]:
df_kr.head()

,video_id,title,publish_time,channel_title,category_id,trending_date,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,tagstring,pred,preproc_pred,pred_en,preprocessed_pred
0,uq5LClQN3cE,안녕하세요 보겸입니다,2020-08-09T09:32:48Z,보겸 BK,24,2020-08-12T00:00:00Z,5947503,53326,105756,139946,https://i.ytimg.com/vi/uq5LClQN3cE/default.jpg,False,False,None,보겸 bokyem,안녕하세요 보겸입니다 보겸 bokyem None,안녕하세요 보겸입니다 보겸 bokyem None,"Hello, it is Bumming Bokyem None",hello bumming bokyem
1,I-ZbZCHsHD0,부락토스의 계획 [총몇명 프리퀄],2020-08-12T09:00:08Z,총몇명,1,2020-08-12T00:00:00Z,963384,28244,494,3339,https://i.ytimg.com/vi/I-ZbZCHsHD0/default.jpg,False,False,"오늘도 정말 감사드립니다!!총몇명 스튜디오 - 총몇명, 십제곱, 5G민, MOVE혁...",총몇명 재밌는 만화 부락토스 루시퍼 총몇명 프리퀄 총몇명 스토리,부락토스의 계획 [총몇명 프리퀄] 총몇명 재밌는 만화 부락토스 루시퍼 총몇명 프리퀄...,부락토스의 계획 총몇명 프리퀄 총몇명 재밌는 만화 부락토스 루시퍼 총몇명 프리퀄...,Subculture's Plan Total Cartoon Ball Total Tot...,subculture plan total cartoon ball total total...
2,3pI_L3-sMVg,안녕하세요 꽈뚜룹입니다.,2020-08-11T15:00:58Z,꽈뚜룹 Quaddurup,24,2020-08-12T00:00:00Z,1743374,36893,1798,8751,https://i.ytimg.com/vi/3pI_L3-sMVg/default.jpg,False,False,앞으로 좀 더 깔끔한 영상제작 약속 드리겠습니다.늘 감사드립니다,꽈뚜룹 한국여행기 quaddurup 뚜룹이 korea southkorea vlog ...,안녕하세요 꽈뚜룹입니다. 꽈뚜룹 한국여행기 quaddurup 뚜룹이 k...,꽈뚜룹입니다 꽈뚜룹 한국여행기 quaddurup 뚜룹이 k...,The pretelogur is a journey to the Korean Sout...,pretelogur journey korean southkorup touper
3,zrsBjYukE8s,박진영 (J.Y. Park) When We Disco (Duet with 선미) M/V,2020-08-11T09:00:13Z,JYP Entertainment,10,2020-08-12T00:00:00Z,3433885,353337,9763,23405,https://i.ytimg.com/vi/zrsBjYukE8s/default.jpg,False,False,MelOn http://kko.to/TWyXd7zYjSpotify https://s...,JYP Entertainment JYP J.Y.Park JYPark 박진영 선미 S...,박진영 (J.Y. Park) When We Disco (Duet with 선미) M...,박진영 J Y Park When We Disco Duet with 선미 M...,JYPJY Park When We Disco Duet withsternMV JYP ...,jypjy park disco duet withsternmv jyp entertai...
4,jbGRowa5tIk,ITZY “Not Shy” M/V TEASER,2020-08-11T15:00:13Z,JYP Entertainment,10,2020-08-12T00:00:00Z,6000070,714301,15176,31040,https://i.ytimg.com/vi/jbGRowa5tIk/default.jpg,False,False,ITZY Not Shy M/V[ITZY Official] https://www.yo...,JYP Entertainment JYP ITZY 있지 ITZY Video ITZY ...,ITZY “Not Shy” M/V TEASER JYP Entertainment JY...,ITZY Not Shy M V TEASER JYP Entertainment JY...,ITZY Not Shy MV TEASER JYP Entertainment JYP I...,itzy shy mv teaser jyp entertainment jyp itzyn...


In [54]:
df_kr['preprocessed_pred'].value_counts()

know                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             11
brbe real time streaming donation support link http toon donate ttuet http toon donate ttuet                                                                                                                                                                                                                                                                                                        

In [55]:
df_kr['pred'].value_counts()

브베 실시간 스트리밍 도네이션 후원 링크 입니당.https://toon.at/donate/ttuet(음성 도네이션만 받고 있어요)브베 페이스북 페이지 친구 해주세용~https://www.facebook.com/브베페이지-2074999436116625                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          9
브베 실시간 스트리밍 투네이션 후원 링크 입니당.https://toon.at/donate/ttuet(음성 도네이션만 

In [56]:
df_kr.drop(columns=['preproc_pred'], inplace = True)

In [57]:
df_kr.shape

(19917, 18)

In [58]:
df_kr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19917 entries, 0 to 19916
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   video_id           19917 non-null  object
 1   title              19917 non-null  object
 2   publish_time       19917 non-null  object
 3   channel_title      19917 non-null  object
 4   category_id        19917 non-null  int64 
 5   trending_date      19917 non-null  object
 6   views              19917 non-null  int64 
 7   likes              19917 non-null  int64 
 8   dislikes           19917 non-null  int64 
 9   comment_count      19917 non-null  int64 
 10  thumbnail_link     19917 non-null  object
 11  comments_disabled  19917 non-null  bool  
 12  ratings_disabled   19917 non-null  bool  
 13  description        19210 non-null  object
 14  tagstring          19917 non-null  object
 15  pred               19917 non-null  object
 16  pred_en            19917 non-null  objec

In [59]:
df_kr.to_feather('/content/drive/MyDrive/Capstone/data/youtube_kr_preprocessed.ftr')